In [67]:
import settings

import time
import datetime
import numpy as np
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username=settings.plotly_name, api_key=settings.plotly_key)
plotly.tools.set_config_file(world_readable=True,sharing='public')

from poloniex import Poloniex
polo = Poloniex()

In [68]:
def loadPolo(pair, period, start, end):
    dates = []
    data={'prices':[],'volumes':[]}
    raw = polo.returnChartData(currencyPair=pair,period=period,start=start,end=end)
    for i in range(len(raw)):
        dates.append( datetime.datetime.fromtimestamp(int(raw[i]['date'])))
        data['prices'].append( float(raw[i]['weightedAverage']))
        data['volumes'].append( float(raw[i]['volume'] ))
    return pd.DataFrame( data, index=dates )

In [69]:
def rollingMean(data,length):
    dates = []
    means = []
    for i in range( length, len(data) ):
        total = 0
        totVol = 0
        for j in range(length):
            date = data.index[i-j]
            total += data.prices[date] * data.volumes[date]
            totVol +=  data.volumes[date]
        means.append( total / totVol )
        dates.append( data.index[i] )
    return pd.Series(means,index=dates)

In [70]:
def macd(data,length1,length2):
    data['mean1'] = rollingMean(data,length1)
    data['mean2'] = rollingMean(data,length2)
    data['macd'] = ( data['mean1'] - data['mean2'] ) / data['mean2']
    return data

In [71]:
def macdTest(data):
    cash = 1.0
    cashes = []
    paid = False
    for i in range(len(data)):
        level = data.iloc[i]['macd']
        if not level: continue
        price = data.iloc[i]['prices']
        if not paid and 0 < level: paid = price
        if paid and 0 > level:
            cash *= 1.0 * price / paid
            paid = False
        cashes.append(cash)
    data['cash']=cashes
    return data

In [72]:
def runMacdTest(pair,period,start,end,length1,length2):
    return macdTest( macd( loadPolo(pair,period,start,end),length1,length2))

In [73]:
def macdTestGraph(data):
    fig = tls.make_subplots(rows=4, cols=1, shared_xaxes=True)
    hover_text = data.apply(lambda r: '<br>'.join(['{}: {}'.format(c, r[c]) for c in data.columns]), axis=1)
    for col in data.columns:
        if col not in ['macd','volumes','cash']:
            fig.append_trace({'x': data.index, 'y': data[col], 'type': 'scatter', 'name': col, 'text': hover_text}, 1, 1)
    fig.append_trace({'x': data.index, 'y': data['cash'], 'type': 'scatter', 'name': 'cash', 'text': hover_text}, 2, 1)
    fig.append_trace({'x': data.index, 'y': data['macd'], 'type': 'bar', 'name': 'macd', 'text': hover_text}, 3, 1)
    fig.append_trace({'x': data.index, 'y': data['volumes'], 'type': 'bar', 'name': 'volumes', 'text': hover_text}, 4, 1)
    return py.iplot(fig, filename='test3')

In [66]:
data = runMacdTest('USDT_BTC',86400,time.mktime(datetime.date(2015,5,1).timetuple()),time.time(),11,21)
macdTestGraph(data)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



In [74]:
def macdHeat(pair,period,start,end,length1,length2, delta):
    data = loadPolo(pair,period,start,end)
    heat = np.full((length2 - length1,length2-length1),1.0)
    for i in range(length1, length2 - delta):
        for j in range( i + delta, length2):
            point = macdTest( macd( data, i, j) )
            heat[i - length1][j - length1] = point.iloc[len(point) - 1]['cash']
    return heat

In [64]:
heat = macdHeat('USDT_BTC',86400,time.mktime(datetime.date(2015,5,1).timetuple()),time.time(),5,50,2)
py.iplot( [ go.Heatmap(z=heat,x=[5,6],y=[7,6]) ] , filename='basic-heatmap')